<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_11_3_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 11: Finetuning**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 11 Material

Module 11: Finetuning

* Part 11.1: Understanding Finetuning [[Video]](https://www.youtube.com/watch?v=fflySydZABM) [[Notebook]](t81_559_class_11_1_finetune.ipynb)
* Part 11.2: Finetuning from the Dashboard [[Video]](https://www.youtube.com/watch?v=RIJj1QLk-V4) [[Notebook]](t81_559_class_11_2_dashboard.ipynb)
* **Part 11.3: Finetuning from Code** [[Video]](https://www.youtube.com/watch?v=29tUrxrneOs) [[Notebook]](t81_559_class_11_3_code.ipynb)
* Part 11.4: Evaluating your Model [[Video]](https://www.youtube.com/watch?v=MrwFSG4PWUY) [[Notebook]](t81_559_class_11_4_eval.ipynb)
* Part 11.5: Finetuning for Text to Image [[Video]](https://www.youtube.com/watch?v=G_FYFSzkB5Y&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_559_class_11_5_image.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [1]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain openai streamlit
    !pip install --upgrade streamlit openai
    !pip install openai==0.28.0

Note: using Google CoLab
  Using cached openai-1.57.4-py3-none-any.whl.metadata (24 kB)
Using cached openai-1.57.4-py3-none-any.whl (390 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0
  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


# Part 11.3: Finetuning from Code

Just like the last part we will utilize the following training data.

* [sarcastic.jsonl](https://data.heatonresearch.com/data/t81-559/finetune/sarcastic.jsonl)
* [sarcastic_val.jsonl](https://data.heatonresearch.com/data/t81-559/finetune/sarcastic_val.jsonl)





In [2]:
import openai

!wget https://data.heatonresearch.com/data/t81-559/finetune/sarcastic.jsonl
!wget https://data.heatonresearch.com/data/t81-559/finetune/sarcastic_val.jsonl
'''
client = OpenAI()

obj = client.files.create(
  file=open("sarcastic.jsonl", "rb"),
  purpose="fine-tune"
)
'''
# Upload the file
obj = openai.File.create(
    file=open("sarcastic.jsonl", "rb"),  # Open the file in binary mode
    purpose="fine-tune"
)


--2024-12-14 13:13:06--  https://data.heatonresearch.com/data/t81-559/finetune/sarcastic.jsonl
Resolving data.heatonresearch.com (data.heatonresearch.com)... 18.160.46.108, 18.160.46.15, 18.160.46.40, ...
Connecting to data.heatonresearch.com (data.heatonresearch.com)|18.160.46.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7202 (7.0K) [binary/octet-stream]
Saving to: ‘sarcastic.jsonl.6’

sarcastic.jsonl.6   100%[===================>]   7.03K  --.-KB/s    in 0s      

2024-12-14 13:13:06 (75.9 MB/s) - ‘sarcastic.jsonl.6’ saved [7202/7202]

--2024-12-14 13:13:06--  https://data.heatonresearch.com/data/t81-559/finetune/sarcastic_val.jsonl
Resolving data.heatonresearch.com (data.heatonresearch.com)... 18.160.46.108, 18.160.46.15, 18.160.46.40, ...
Connecting to data.heatonresearch.com (data.heatonresearch.com)|18.160.46.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2575 (2.5K) [binary/octet-stream]
Saving to: ‘sarcastic_val.js

In [3]:
obj.id

'file-4iuLwUcic8TqaB5D5oJDNK'

In [11]:
print("Fine-Tuning Job Status:", obj.status)

Fine-Tuning Job Status: processed


### Run and Monitor Finetuning

In [19]:
import time
import openai
'''
# Start the fine-tuning job
train = client.fine_tuning.jobs.create(
    training_file=obj.id,
    model="gpt-4o-mini-2024-07-18"
)
'''
train = openai.FineTuningJob.create(
    training_file=obj.id,
    model="gpt-4o-mini-2024-07-18"
)

fine_tune_job_id = train["id"]
print("Fine-Tuning Job ID:", fine_tune_job_id)


done = False

# Initialize a set to store processed event IDs
processed_event_ids = set()

while not done:
    '''
    # Retrieve the latest status of the fine-tuning job
    status = client.fine_tuning.jobs.retrieve(train.id)
    print(f"Job status: {status.status}")
    '''
    # Step 3: Retrieve the fine-tuning job status
    status_response = openai.FineTuningJob.retrieve(id=fine_tune_job_id)
    status = status_response["status"]
    print("Fine-Tuning Job Status:", status)
    '''
    # Fetch all events related to the fine-tuning job
    events = client.fine_tuning.jobs.list_events(fine_tuning_job_id=train.id)
    '''
    # Retrieve the events for the fine-tuning job
    events_response = openai.FineTuningJob.list_events(id=fine_tune_job_id)

    # Extract the list of events
    events = events_response["data"]


    # Collect new events that haven't been processed yet
    new_events = []
    '''
    for event in events:
        if event.id not in processed_event_ids:
            new_events.append(event)
            processed_event_ids.add(event.id)
    '''
    for event in events:
        if event["id"] not in processed_event_ids:  # Use event["id"] instead of event.id
            new_events.append(event)
            processed_event_ids.add(event["id"])  # Add the ID to the processed set
    # Sort the new events in chronological order
    new_events.sort(key=lambda e: e.created_at)

    # Display the new events in order
    for event in new_events:
        print(f"{event.created_at}: {event.message}")

    if status == "succeeded":
        done = True
        print("Done!")
    elif status == "failed":
        done = True
        print("Failed!")
    else:
        print("Waiting for updates...")
        time.sleep(20)  # Sleep for 20 seconds


Fine-Tuning Job ID: ftjob-eb5tUUfIqNnI7VTK6YsCVFMd
Fine-Tuning Job Status: validating_files
1734182815: Validating training file: file-4iuLwUcic8TqaB5D5oJDNK
1734182815: Created fine-tuning job: ftjob-eb5tUUfIqNnI7VTK6YsCVFMd
Waiting for updates...
Fine-Tuning Job Status: running
1734182819: Fine-tuning job started
1734182819: Files validated, moving job to queued state
Waiting for updates...
Fine-Tuning Job Status: running
Waiting for updates...
Fine-Tuning Job Status: running
Waiting for updates...
Fine-Tuning Job Status: running
Waiting for updates...
Fine-Tuning Job Status: running
Waiting for updates...
Fine-Tuning Job Status: running
Waiting for updates...
Fine-Tuning Job Status: running
1734182951: Step 1/84: training loss=3.57
Waiting for updates...
Fine-Tuning Job Status: running
1734182953: Step 3/84: training loss=3.05
1734182956: Step 6/84: training loss=2.44
1734182956: Step 5/84: training loss=2.62
1734182956: Step 4/84: training loss=3.66
1734182958: Step 8/84: training 

In [21]:
# Retrieve the fine-tuning job details
fine_tuning_job = openai.FineTuningJob.retrieve(id=fine_tune_job_id)

# Access the fine-tuned model ID
model_id = fine_tuning_job.get("fine_tuned_model")

if model_id:
    print(f"Trained model ID: {model_id}")
else:
    print("The fine-tuning job has not completed or no model ID is available.")


Trained model ID: ft:gpt-4o-mini-2024-07-18:personal::AeMWnZUP


### Test the Finetuned Model

In [23]:
import openai
# Call OpenAI API to get the response
completion = openai.ChatCompletion.create(
  model=model_id,
  messages=[
    {"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."},
    {"role": "user", "content": "What is the capital of the USA?"}
  ]
)

print(completion.choices[0].message)

{
  "role": "assistant",
  "content": "Washington, D.C., as if everyone doesn\u2019t know that already.",
  "refusal": null
}


### Delete Old Models

In [ ]:
#client.models.delete("ft:gpt-4o-mini-2024-07-18:personal:sarcastic:A9yCtR0b")